In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [3]:
!ls

sample_data  spark-2.4.4-bin-hadoop2.7.tgz


In [4]:
!tar -xvf spark-2.4.4-bin-hadoop2.7.tgz

spark-2.4.4-bin-hadoop2.7/
spark-2.4.4-bin-hadoop2.7/R/
spark-2.4.4-bin-hadoop2.7/R/lib/
spark-2.4.4-bin-hadoop2.7/R/lib/sparkr.zip
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/R.css
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/00Index.html
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/aliases.rds
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/AnIndex
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/SparkR.rdx
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/SparkR.rdb
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/paths.rds
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/worker.R
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/daemon.R
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/testthat/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/testthat/te

In [5]:
!ls 

sample_data  spark-2.4.4-bin-hadoop2.7	spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
df = spark.createDataFrame([{"Google": "Colab","Spark": "Scala"} ,{"Google": "Dataproc","Spark":"Python"}])
df.show()

/content/spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+------+
|  Google| Spark|
+--------+------+
|   Colab| Scala|
|Dataproc|Python|
+--------+------+



In [0]:
import string # to remove punctuation
import re
import pandas as pd
import os
import itertools
from collections import Counter
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import udf, col


# Step 1: Load data

In [11]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
!ls

drive  sample_data  spark-2.4.4-bin-hadoop2.7  spark-2.4.4-bin-hadoop2.7.tgz


In [21]:
#storing csv as dataframes
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
dataset = spark.read.csv('/content/drive/My Drive/tick_data.csv',inferSchema=True, header =True)
type(dataset)

pyspark.sql.dataframe.DataFrame

In [94]:
dataset.printSchema()

root
 |-- DATE: integer (nullable = true)
 |-- TIME_S: string (nullable = true)
 |-- SYM_ROOT: string (nullable = true)
 |-- SIZE: integer (nullable = true)
 |-- TRADE: double (nullable = true)



In [95]:
dataset.show()

+--------+--------+--------+----+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|
+--------+--------+--------+----+------+
|20120103|09:04:08|    AAPL| 500|409.21|
|20120103|09:06:49|    AAPL| 100|409.41|
|20120103|09:21:10|    AAPL| 100|409.49|
|20120103|09:26:28|    AAPL| 100|409.04|
|20120103|09:29:55|    AAPL| 100| 410.3|
|20120103|09:30:00|    AAPL| 200|409.91|
|20120103|09:30:04|    AAPL| 100|410.11|
|20120103|09:30:04|    AAPL| 200|410.05|
|20120103|09:30:06|    AAPL| 100| 410.2|
|20120103|09:30:20|    AAPL| 140| 409.9|
|20120103|09:30:39|    AAPL| 100|409.82|
|20120103|09:30:49|    AAPL| 100|409.76|
|20120103|09:30:50|    AAPL| 100| 409.8|
|20120103|09:30:50|    AAPL| 200| 409.8|
|20120103|09:31:04|    AAPL| 150|409.53|
|20120103|09:31:25|    AAPL| 174|409.79|
|20120103|09:31:50|    AAPL| 100|409.48|
|20120103|09:31:55|    AAPL| 200|409.44|
|20120103|09:32:29|    AAPL| 100|409.95|
|20120103|09:33:13|    AAPL| 400|410.16|
+--------+--------+--------+----+------+
only showing top

# Step 2: Data type conversions

In [0]:
from pyspark.sql.functions import col, count, hour, sum

In [0]:
df = dataset.select('DATE', 'TIME_S', 'SYM_ROOT', 'SIZE', 'TRADE')
df.show()

In [0]:
#converting date to string type
dfs = df.withColumn("DATE", df["DATE"].cast(StringType()))

In [0]:
#creating a new column for timestamp
from pyspark.sql.functions import unix_timestamp

dfs = df.withColumn("parsed", to_timestamp("TIME_S", "HH:mm:ss"))

In [100]:
dfs.show()

+--------+--------+--------+----+------+-------------------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|
+--------+--------+--------+----+------+-------------------+
|20120103|09:04:08|    AAPL| 500|409.21|1970-01-01 09:04:08|
|20120103|09:06:49|    AAPL| 100|409.41|1970-01-01 09:06:49|
|20120103|09:21:10|    AAPL| 100|409.49|1970-01-01 09:21:10|
|20120103|09:26:28|    AAPL| 100|409.04|1970-01-01 09:26:28|
|20120103|09:29:55|    AAPL| 100| 410.3|1970-01-01 09:29:55|
|20120103|09:30:00|    AAPL| 200|409.91|1970-01-01 09:30:00|
|20120103|09:30:04|    AAPL| 100|410.11|1970-01-01 09:30:04|
|20120103|09:30:04|    AAPL| 200|410.05|1970-01-01 09:30:04|
|20120103|09:30:06|    AAPL| 100| 410.2|1970-01-01 09:30:06|
|20120103|09:30:20|    AAPL| 140| 409.9|1970-01-01 09:30:20|
|20120103|09:30:39|    AAPL| 100|409.82|1970-01-01 09:30:39|
|20120103|09:30:49|    AAPL| 100|409.76|1970-01-01 09:30:49|
|20120103|09:30:50|    AAPL| 100| 409.8|1970-01-01 09:30:50|
|20120103|09:30:50|    A

In [101]:
#Extracting Hour from TIME_S
d1 = dfs.withColumn('Time_H', hour('TIME_S'))
d1.show()

+--------+--------+--------+----+------+-------------------+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+--------+--------+--------+----+------+-------------------+------+
|20120103|09:04:08|    AAPL| 500|409.21|1970-01-01 09:04:08|     9|
|20120103|09:06:49|    AAPL| 100|409.41|1970-01-01 09:06:49|     9|
|20120103|09:21:10|    AAPL| 100|409.49|1970-01-01 09:21:10|     9|
|20120103|09:26:28|    AAPL| 100|409.04|1970-01-01 09:26:28|     9|
|20120103|09:29:55|    AAPL| 100| 410.3|1970-01-01 09:29:55|     9|
|20120103|09:30:00|    AAPL| 200|409.91|1970-01-01 09:30:00|     9|
|20120103|09:30:04|    AAPL| 100|410.11|1970-01-01 09:30:04|     9|
|20120103|09:30:04|    AAPL| 200|410.05|1970-01-01 09:30:04|     9|
|20120103|09:30:06|    AAPL| 100| 410.2|1970-01-01 09:30:06|     9|
|20120103|09:30:20|    AAPL| 140| 409.9|1970-01-01 09:30:20|     9|
|20120103|09:30:39|    AAPL| 100|409.82|1970-01-01 09:30:39|     9|
|20120103|09:30:49|    AAPL| 100|409.76|1970-01-

# Step 3: Calculate total trading volume for each stock in a certain hour

In [134]:
aggs = d1.orderBy('SYM_ROOT', ascending=True).groupBy("DATE", "SYM_ROOT", "Time_H").sum("SIZE")
aggs1 = aggs.select(col("DATE").alias("DATEE"), col("SYM_ROOT").alias("SYM_ROOTT"), col("Time_H").alias("Time_HH"), col("sum(SIZE)").alias("Size_H"))
aggs1.show()

+--------+---------+-------+------+
|   DATEE|SYM_ROOTT|Time_HH|Size_H|
+--------+---------+-------+------+
|20120103|     AAPL|      9| 21092|
|20120103|     AAPL|     10| 12041|
|20120103|     AAPL|     11|  9473|
|20120103|     AAPL|     12|  5718|
|20120103|     AAPL|     13|  8103|
|20120103|     AAPL|     14|  7783|
|20120103|     AAPL|     15| 15334|
|20120103|     GOOG|      9|  8314|
|20120103|     GOOG|     10|  8080|
|20120103|     GOOG|     11|  6776|
|20120103|     GOOG|     12|  6619|
|20120103|     GOOG|     13|  6517|
|20120103|     GOOG|     14|  8453|
|20120103|     GOOG|     15| 17496|
|20120103|      IBM|      9| 14064|
|20120103|      IBM|     10| 12055|
|20120103|      IBM|     11| 12863|
|20120103|      IBM|     12| 11694|
|20120103|      IBM|     13|  5109|
|20120103|      IBM|     14|  9033|
+--------+---------+-------+------+
only showing top 20 rows



# Step 4: Calculate hourly return of each stock

In [103]:
#finding the last entry in an hour for each company's stock
dfma = d1.groupBy("SYM_ROOT", "Time_H").agg(max("parsed").alias('paa')).orderBy('SYM_ROOT','Time_H', ascending=True)
dfma.show()

+--------+------+-------------------+
|SYM_ROOT|Time_H|                paa|
+--------+------+-------------------+
|    AAPL|     9|1970-01-01 09:59:14|
|    AAPL|    10|1970-01-01 10:59:12|
|    AAPL|    11|1970-01-01 11:53:49|
|    AAPL|    12|1970-01-01 12:59:58|
|    AAPL|    13|1970-01-01 13:56:26|
|    AAPL|    14|1970-01-01 14:59:56|
|    AAPL|    15|1970-01-01 15:59:59|
|    GOOG|     9|1970-01-01 09:59:46|
|    GOOG|    10|1970-01-01 10:59:29|
|    GOOG|    11|1970-01-01 11:59:44|
|    GOOG|    12|1970-01-01 12:59:25|
|    GOOG|    13|1970-01-01 13:59:21|
|    GOOG|    14|1970-01-01 14:59:24|
|    GOOG|    15|1970-01-01 15:59:55|
|     IBM|     9|1970-01-01 09:58:30|
|     IBM|    10|1970-01-01 10:59:45|
|     IBM|    11|1970-01-01 11:59:42|
|     IBM|    12|1970-01-01 12:59:37|
|     IBM|    13|1970-01-01 13:58:41|
|     IBM|    14|1970-01-01 14:59:09|
+--------+------+-------------------+
only showing top 20 rows



In [104]:
#changing column names
dfma1 = dfma.select(col("SYM_ROOT").alias("sys"), col("Time_H").alias("H"), "paa")
dfma1.show()

+----+---+-------------------+
| sys|  H|                paa|
+----+---+-------------------+
|AAPL|  9|1970-01-01 09:59:14|
|AAPL| 10|1970-01-01 10:59:12|
|AAPL| 11|1970-01-01 11:53:49|
|AAPL| 12|1970-01-01 12:59:58|
|AAPL| 13|1970-01-01 13:56:26|
|AAPL| 14|1970-01-01 14:59:56|
|AAPL| 15|1970-01-01 15:59:59|
|GOOG|  9|1970-01-01 09:59:46|
|GOOG| 10|1970-01-01 10:59:29|
|GOOG| 11|1970-01-01 11:59:44|
|GOOG| 12|1970-01-01 12:59:25|
|GOOG| 13|1970-01-01 13:59:21|
|GOOG| 14|1970-01-01 14:59:24|
|GOOG| 15|1970-01-01 15:59:55|
| IBM|  9|1970-01-01 09:58:30|
| IBM| 10|1970-01-01 10:59:45|
| IBM| 11|1970-01-01 11:59:42|
| IBM| 12|1970-01-01 12:59:37|
| IBM| 13|1970-01-01 13:58:41|
| IBM| 14|1970-01-01 14:59:09|
+----+---+-------------------+
only showing top 20 rows



In [105]:
#joining dfma1 and d1
inner_join_max = dfma1.join(d1, dfma1.paa == d1.parsed)
inner_join_max.show()

+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
| sys|  H|                paa|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
|AAPL|  9|1970-01-01 09:59:14|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|
|AAPL| 10|1970-01-01 10:59:12|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|
|AAPL| 11|1970-01-01 11:53:49|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|
|AAPL| 12|1970-01-01 12:59:58|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|
|AAPL| 13|1970-01-01 13:56:26|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|
|MSFT| 14|1970-01-01 14:59:55|20120103|14:59:55|    AAPL| 200|412.11|1970-01-01 14:59:55|    14|
|AAPL| 14|1970-01-01 14:59:56|20120103|14:59:56|    AAPL| 100|412.16|1970-01-01 14:59:56|    14|
| IBM| 15|1970-01-01 15:59:55|

In [106]:
#keeping the values of rows when both 'sys' and 'SYM_ROOT' match
inn_max =  inner_join_max.filter(inner_join_max.sys == inner_join_max.SYM_ROOT)
inn_max.show()

+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
| sys|  H|                paa|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
|AAPL|  9|1970-01-01 09:59:14|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|
|AAPL| 10|1970-01-01 10:59:12|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|
|AAPL| 11|1970-01-01 11:53:49|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|
|AAPL| 12|1970-01-01 12:59:58|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|
|AAPL| 13|1970-01-01 13:56:26|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|
|AAPL| 14|1970-01-01 14:59:56|20120103|14:59:56|    AAPL| 100|412.16|1970-01-01 14:59:56|    14|
|AAPL| 15|1970-01-01 15:59:59|20120103|15:59:59|    AAPL| 100|411.45|1970-01-01 15:59:59|    15|
|AAPL| 15|1970-01-01 15:59:59|

In [0]:
#dropping duplicates based on the time and stock name
import pyspark.sql.functions as F
inn_max1 = inn_max.orderBy('SYM_ROOT', F.col('parsed').desc()).dropDuplicates(subset = ['parsed', 'SYM_ROOT'])

In [109]:
inn_max2 = inn_max1.orderBy('SYM_ROOT', 'Time_H', ascending=True)
inn_max2.show()

+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
| sys|  H|                paa|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
|AAPL|  9|1970-01-01 09:59:14|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|
|AAPL| 10|1970-01-01 10:59:12|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|
|AAPL| 11|1970-01-01 11:53:49|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|
|AAPL| 12|1970-01-01 12:59:58|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|
|AAPL| 13|1970-01-01 13:56:26|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|
|AAPL| 14|1970-01-01 14:59:56|20120103|14:59:56|    AAPL| 100|412.16|1970-01-01 14:59:56|    14|
|AAPL| 15|1970-01-01 15:59:59|20120103|15:59:59|    AAPL| 100|411.45|1970-01-01 15:59:59|    15|
|GOOG|  9|1970-01-01 09:59:46|

In [110]:
#dataframe with maximum time for each hour per company stock is obtained
dfmax = inn_max2.drop('sys', 'H', 'paa')
dfmax.show()

+--------+--------+--------+----+------+-------------------+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+--------+--------+--------+----+------+-------------------+------+
|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|
|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|
|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|
|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|
|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|
|20120103|14:59:56|    AAPL| 100|412.16|1970-01-01 14:59:56|    14|
|20120103|15:59:59|    AAPL| 100|411.45|1970-01-01 15:59:59|    15|
|20120103|09:59:46|    GOOG| 100|659.43|1970-01-01 09:59:46|     9|
|20120103|10:59:29|    GOOG| 100|659.95|1970-01-01 10:59:29|    10|
|20120103|11:59:44|    GOOG| 100|661.86|1970-01-01 11:59:44|    11|
|20120103|12:59:25|    GOOG| 100|659.56|1970-01-01 12:59:25|    12|
|20120103|13:59:21|    GOOG| 100|660.17|1970-01-

In [111]:
dfmax.count()

28

In [112]:
#finding the first entry in an hour for each company's stock
dfmi = d1.groupBy("SYM_ROOT", "Time_H").agg(min("parsed").alias('paa')).orderBy('SYM_ROOT','Time_H', ascending=True)
dfmi.show()

+--------+------+-------------------+
|SYM_ROOT|Time_H|                paa|
+--------+------+-------------------+
|    AAPL|     9|1970-01-01 09:04:08|
|    AAPL|    10|1970-01-01 10:00:08|
|    AAPL|    11|1970-01-01 11:01:51|
|    AAPL|    12|1970-01-01 12:00:57|
|    AAPL|    13|1970-01-01 13:00:13|
|    AAPL|    14|1970-01-01 14:00:01|
|    AAPL|    15|1970-01-01 15:00:12|
|    GOOG|     9|1970-01-01 09:05:35|
|    GOOG|    10|1970-01-01 10:00:06|
|    GOOG|    11|1970-01-01 11:03:35|
|    GOOG|    12|1970-01-01 12:01:37|
|    GOOG|    13|1970-01-01 13:00:31|
|    GOOG|    14|1970-01-01 14:04:00|
|    GOOG|    15|1970-01-01 15:00:00|
|     IBM|     9|1970-01-01 09:30:03|
|     IBM|    10|1970-01-01 10:00:03|
|     IBM|    11|1970-01-01 11:00:35|
|     IBM|    12|1970-01-01 12:00:49|
|     IBM|    13|1970-01-01 13:00:53|
|     IBM|    14|1970-01-01 14:00:26|
+--------+------+-------------------+
only showing top 20 rows



In [113]:
#changing column names
dfmi1 = dfmi.select(col("SYM_ROOT").alias("sys"), col("Time_H").alias("H"), "paa")
dfmi1.show()

+----+---+-------------------+
| sys|  H|                paa|
+----+---+-------------------+
|AAPL|  9|1970-01-01 09:04:08|
|AAPL| 10|1970-01-01 10:00:08|
|AAPL| 11|1970-01-01 11:01:51|
|AAPL| 12|1970-01-01 12:00:57|
|AAPL| 13|1970-01-01 13:00:13|
|AAPL| 14|1970-01-01 14:00:01|
|AAPL| 15|1970-01-01 15:00:12|
|GOOG|  9|1970-01-01 09:05:35|
|GOOG| 10|1970-01-01 10:00:06|
|GOOG| 11|1970-01-01 11:03:35|
|GOOG| 12|1970-01-01 12:01:37|
|GOOG| 13|1970-01-01 13:00:31|
|GOOG| 14|1970-01-01 14:04:00|
|GOOG| 15|1970-01-01 15:00:00|
| IBM|  9|1970-01-01 09:30:03|
| IBM| 10|1970-01-01 10:00:03|
| IBM| 11|1970-01-01 11:00:35|
| IBM| 12|1970-01-01 12:00:49|
| IBM| 13|1970-01-01 13:00:53|
| IBM| 14|1970-01-01 14:00:26|
+----+---+-------------------+
only showing top 20 rows



In [114]:
#joining dfmi1 and d1
inner_join_min = dfmi1.join(d1, dfmi1.paa == d1.parsed)
inner_join_min.show()

+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
| sys|  H|                paa|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
|AAPL|  9|1970-01-01 09:04:08|20120103|09:04:08|    AAPL| 500|409.21|1970-01-01 09:04:08|     9|
|AAPL| 10|1970-01-01 10:00:08|20120103|10:00:08|    AAPL| 100|411.68|1970-01-01 10:00:08|    10|
|AAPL| 11|1970-01-01 11:01:51|20120103|11:01:51|    AAPL| 100|411.36|1970-01-01 11:01:51|    11|
|AAPL| 12|1970-01-01 12:00:57|20120103|12:00:57|    AAPL| 290|411.22|1970-01-01 12:00:57|    12|
|AAPL| 13|1970-01-01 13:00:13|20120103|13:00:13|    AAPL| 100|410.26|1970-01-01 13:00:13|    13|
|AAPL| 14|1970-01-01 14:00:01|20120103|14:00:01|    AAPL| 800|411.31|1970-01-01 14:00:01|    14|
|AAPL| 15|1970-01-01 15:00:12|20120103|15:00:12|    AAPL| 100|412.17|1970-01-01 15:00:12|    15|
|GOOG|  9|1970-01-01 09:05:35|

In [115]:
#keeping the values of rows when both 'sys' and 'SYM_ROOT' match
inn_min =  inner_join_min.filter(inner_join_min.sys == inner_join_min.SYM_ROOT)
inn_min.show()

+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
| sys|  H|                paa|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
|AAPL|  9|1970-01-01 09:04:08|20120103|09:04:08|    AAPL| 500|409.21|1970-01-01 09:04:08|     9|
|AAPL| 10|1970-01-01 10:00:08|20120103|10:00:08|    AAPL| 100|411.68|1970-01-01 10:00:08|    10|
|AAPL| 11|1970-01-01 11:01:51|20120103|11:01:51|    AAPL| 100|411.36|1970-01-01 11:01:51|    11|
|AAPL| 12|1970-01-01 12:00:57|20120103|12:00:57|    AAPL| 290|411.22|1970-01-01 12:00:57|    12|
|AAPL| 13|1970-01-01 13:00:13|20120103|13:00:13|    AAPL| 100|410.26|1970-01-01 13:00:13|    13|
|AAPL| 14|1970-01-01 14:00:01|20120103|14:00:01|    AAPL| 800|411.31|1970-01-01 14:00:01|    14|
|AAPL| 15|1970-01-01 15:00:12|20120103|15:00:12|    AAPL| 100|412.17|1970-01-01 15:00:12|    15|
|GOOG|  9|1970-01-01 09:05:35|

In [0]:
#dropping duplicates based on the time and stock name
import pyspark.sql.functions as F
inn_min1 = inn_min.orderBy('SYM_ROOT', F.col('parsed').desc()).dropDuplicates(subset = ['parsed', 'SYM_ROOT'])

In [117]:
inn_min2 = inn_min1.orderBy('SYM_ROOT', 'Time_H', ascending=True)
inn_min2.show()

+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
| sys|  H|                paa|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+----+---+-------------------+--------+--------+--------+----+------+-------------------+------+
|AAPL|  9|1970-01-01 09:04:08|20120103|09:04:08|    AAPL| 500|409.21|1970-01-01 09:04:08|     9|
|AAPL| 10|1970-01-01 10:00:08|20120103|10:00:08|    AAPL| 100|411.68|1970-01-01 10:00:08|    10|
|AAPL| 11|1970-01-01 11:01:51|20120103|11:01:51|    AAPL| 100|411.36|1970-01-01 11:01:51|    11|
|AAPL| 12|1970-01-01 12:00:57|20120103|12:00:57|    AAPL| 290|411.22|1970-01-01 12:00:57|    12|
|AAPL| 13|1970-01-01 13:00:13|20120103|13:00:13|    AAPL| 100|410.26|1970-01-01 13:00:13|    13|
|AAPL| 14|1970-01-01 14:00:01|20120103|14:00:01|    AAPL| 800|411.31|1970-01-01 14:00:01|    14|
|AAPL| 15|1970-01-01 15:00:12|20120103|15:00:12|    AAPL| 100|412.17|1970-01-01 15:00:12|    15|
|GOOG|  9|1970-01-01 09:05:35|

In [152]:
#dataframe with minimum time for each hour per company stock is obtained
dfmin = inn_min2.drop('sys', 'H', 'paa')
dfmin.show()

+--------+--------+--------+----+------+-------------------+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+--------+--------+--------+----+------+-------------------+------+
|20120103|09:04:08|    AAPL| 500|409.21|1970-01-01 09:04:08|     9|
|20120103|10:00:08|    AAPL| 100|411.68|1970-01-01 10:00:08|    10|
|20120103|11:01:51|    AAPL| 100|411.36|1970-01-01 11:01:51|    11|
|20120103|12:00:57|    AAPL| 290|411.22|1970-01-01 12:00:57|    12|
|20120103|13:00:13|    AAPL| 100|410.26|1970-01-01 13:00:13|    13|
|20120103|14:00:01|    AAPL| 800|411.31|1970-01-01 14:00:01|    14|
|20120103|15:00:12|    AAPL| 100|412.17|1970-01-01 15:00:12|    15|
|20120103|09:05:35|    GOOG| 100|655.46|1970-01-01 09:05:35|     9|
|20120103|10:00:06|    GOOG| 100|659.42|1970-01-01 10:00:06|    10|
|20120103|11:03:35|    GOOG| 100|659.74|1970-01-01 11:03:35|    11|
|20120103|12:01:37|    GOOG| 100|662.06|1970-01-01 12:01:37|    12|
|20120103|13:00:31|    GOOG| 100|660.16|1970-01-

In [153]:
dfmin.count()

28

In [0]:
#combining dfmin and dfmax
dcomb = dfmin.union(dfmax)

In [155]:
#ordering it based on stock name and time
dcomb = dcomb.orderBy('SYM_ROOT', 'parsed', ascending=True)
dcomb.show()

+--------+--------+--------+----+------+-------------------+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|
+--------+--------+--------+----+------+-------------------+------+
|20120103|09:04:08|    AAPL| 500|409.21|1970-01-01 09:04:08|     9|
|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|
|20120103|10:00:08|    AAPL| 100|411.68|1970-01-01 10:00:08|    10|
|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|
|20120103|11:01:51|    AAPL| 100|411.36|1970-01-01 11:01:51|    11|
|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|
|20120103|12:00:57|    AAPL| 290|411.22|1970-01-01 12:00:57|    12|
|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|
|20120103|13:00:13|    AAPL| 100|410.26|1970-01-01 13:00:13|    13|
|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|
|20120103|14:00:01|    AAPL| 800|411.31|1970-01-01 14:00:01|    14|
|20120103|14:59:56|    AAPL| 100|412.16|1970-01-

In [156]:
dcomb.count()

56

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [0]:
#registering dataframe as table
sqlContext.registerDataFrameAsTable(dcomb, "dcomble")

In [159]:
#using lag function to create a new column 'prev' for calculation purposes
dcalc = sqlContext.sql("""select *, lag(TRADE, 1, 0) over (partition by SYM_ROOT, Time_H order by parsed) as prev from dcomble""")
dcalc1 = dcalc.orderBy('SYM_ROOT', 'parsed', ascending=True)
dcalc1.show()

+--------+--------+--------+----+------+-------------------+------+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|  prev|
+--------+--------+--------+----+------+-------------------+------+------+
|20120103|09:04:08|    AAPL| 500|409.21|1970-01-01 09:04:08|     9|   0.0|
|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|409.21|
|20120103|10:00:08|    AAPL| 100|411.68|1970-01-01 10:00:08|    10|   0.0|
|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|411.68|
|20120103|11:01:51|    AAPL| 100|411.36|1970-01-01 11:01:51|    11|   0.0|
|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|411.36|
|20120103|12:00:57|    AAPL| 290|411.22|1970-01-01 12:00:57|    12|   0.0|
|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|411.22|
|20120103|13:00:13|    AAPL| 100|410.26|1970-01-01 13:00:13|    13|   0.0|
|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|410.26|
|20120103|14:00:01|    AA

In [160]:
#filtering rows in column 'prev' with values 0.0
dcalc2 =  dcalc1.filter(dcalc1.prev != 0.0)
dcalc2.show()

+--------+--------+--------+----+------+-------------------+------+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|  prev|
+--------+--------+--------+----+------+-------------------+------+------+
|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|409.21|
|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|411.68|
|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|411.36|
|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|411.22|
|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|410.26|
|20120103|14:59:56|    AAPL| 100|412.16|1970-01-01 14:59:56|    14|411.31|
|20120103|15:59:59|    AAPL| 100|411.45|1970-01-01 15:59:59|    15|412.17|
|20120103|09:59:46|    GOOG| 100|659.43|1970-01-01 09:59:46|     9|655.46|
|20120103|10:59:29|    GOOG| 100|659.95|1970-01-01 10:59:29|    10|659.42|
|20120103|11:59:44|    GOOG| 100|661.86|1970-01-01 11:59:44|    11|659.74|
|20120103|12:59:25|    GO

In [161]:
#FORMULA for RETURN: return = (price of last trade - price of first trade)/price of first trade
#finding the difference
dcalc3 = dcalc2.withColumn('difference', dcalc2['TRADE'] - dcalc2['prev'])
dcalc3.show()

+--------+--------+--------+----+------+-------------------+------+------+--------------------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|  prev|          difference|
+--------+--------+--------+----+------+-------------------+------+------+--------------------+
|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|409.21|   2.170000000000016|
|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|411.68|-0.44999999999998863|
|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|411.36|-0.06000000000000...|
|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|411.22| -1.0300000000000296|
|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|410.26|  0.9499999999999886|
|20120103|14:59:56|    AAPL| 100|412.16|1970-01-01 14:59:56|    14|411.31|  0.8500000000000227|
|20120103|15:59:59|    AAPL| 100|411.45|1970-01-01 15:59:59|    15|412.17| -0.7200000000000273|
|20120103|09:59:46|    GOOG| 100|659.43|

In [162]:
#finding hourly return
dcalc4 = dcalc3.withColumn('return', (F.col('difference') / F.col('prev')))
dcalc4.show()

+--------+--------+--------+----+------+-------------------+------+------+--------------------+--------------------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|  prev|          difference|              return|
+--------+--------+--------+----+------+-------------------+------+------+--------------------+--------------------+
|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|409.21|   2.170000000000016|0.005302900711126356|
|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|411.68|-0.44999999999998863|-0.00109308200544...|
|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|411.36|-0.06000000000000...|-1.45857642940495...|
|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|411.22| -1.0300000000000296|-0.00250474198725...|
|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|410.26|  0.9499999999999886|0.002315604738458...|
|20120103|14:59:56|    AAPL| 100|412.16|1970-01-01 14:59:56|    

In [163]:
dcalc5 = dcalc4.join(aggs1, dcalc4.Time_H == aggs1.Time_HH)
dcalc5 =  dcalc5.filter(dcalc5.SYM_ROOT == dcalc5.SYM_ROOTT)

dcalc5.drop('DATEE', 'SYM_ROOTT', 'Time_HH').show()

+--------+--------+--------+----+------+-------------------+------+------+--------------------+--------------------+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|             parsed|Time_H|  prev|          difference|              return|Size_H|
+--------+--------+--------+----+------+-------------------+------+------+--------------------+--------------------+------+
|20120103|09:59:14|    AAPL| 200|411.38|1970-01-01 09:59:14|     9|409.21|   2.170000000000016|0.005302900711126356| 21092|
|20120103|10:59:12|    AAPL| 100|411.23|1970-01-01 10:59:12|    10|411.68|-0.44999999999998863|-0.00109308200544...| 12041|
|20120103|11:53:49|    AAPL| 100| 411.3|1970-01-01 11:53:49|    11|411.36|-0.06000000000000...|-1.45857642940495...|  9473|
|20120103|12:59:58|    AAPL| 100|410.19|1970-01-01 12:59:58|    12|411.22| -1.0300000000000296|-0.00250474198725...|  5718|
|20120103|13:56:26|    AAPL| 100|411.21|1970-01-01 13:56:26|    13|410.26|  0.9499999999999886|0.002315604738458...|  8103|
|2012010

# Step 5: Store the sorted the output in a csv file

In [164]:
dcalc5.printSchema()


root
 |-- DATE: integer (nullable = true)
 |-- TIME_S: string (nullable = true)
 |-- SYM_ROOT: string (nullable = true)
 |-- SIZE: integer (nullable = true)
 |-- TRADE: double (nullable = true)
 |-- parsed: timestamp (nullable = true)
 |-- Time_H: integer (nullable = true)
 |-- prev: double (nullable = true)
 |-- difference: double (nullable = true)
 |-- return: double (nullable = true)
 |-- DATEE: integer (nullable = true)
 |-- SYM_ROOTT: string (nullable = true)
 |-- Time_HH: integer (nullable = true)
 |-- Size_H: long (nullable = true)



In [0]:
dcalc6 = dcalc5.withColumn("DATE", dcalc4["DATE"].cast(StringType()))

In [0]:
dcalc7 = dcalc6.withColumn("TRADE", dcalc5["TRADE"].cast(StringType()))
dcalc8 = dcalc7.withColumn("return", dcalc6["return"].cast(StringType()))
dcalc9 = dcalc8.withColumn("Time_H", dcalc6["Time_H"].cast(StringType()))
dcalc10 = dcalc9.withColumn("Size_H", dcalc6["Size_H"].cast(StringType()))

In [167]:
#final output
dcalc11 = dcalc10.select('SYM_ROOT', 'DATE', 'Time_H', 'Size_H', 'return')
dcalc11.show()

+--------+--------+------+------+--------------------+
|SYM_ROOT|    DATE|Time_H|Size_H|              return|
+--------+--------+------+------+--------------------+
|    AAPL|20120103|     9| 21092|0.005302900711126356|
|    AAPL|20120103|    10| 12041|-0.00109308200544...|
|    AAPL|20120103|    11|  9473|-1.45857642940495...|
|    AAPL|20120103|    12|  5718|-0.00250474198725...|
|    AAPL|20120103|    13|  8103|0.002315604738458...|
|    AAPL|20120103|    14|  7783|0.002066567795580...|
|    AAPL|20120103|    15| 15334|-0.00174685202707...|
|    GOOG|20120103|     9|  8314|0.006056815061178277|
|    GOOG|20120103|    10|  8080|8.037366170272154E-4|
|    GOOG|20120103|    11|  6776|0.003213387091884...|
|    GOOG|20120103|    12|  6619|-0.00377609280125...|
|    GOOG|20120103|    13|  6517|1.514784294715054...|
|    GOOG|20120103|    14|  8453|0.006965897389302839|
|    GOOG|20120103|    15| 17496|9.169071669071874E-4|
|     IBM|20120103|     9| 14064|0.003090531251665224|
|     IBM|

In [0]:
dcalc11.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("/content/drive/My Drive/anedun_HW3.csv")